# Theophylline NCA Analysis with pyNCA

This notebook demonstrates a complete Non-Compartmental Analysis (NCA) workflow using pyNCA with the theophylline dataset.

In [ ]:
import pandas as pd
import numpy as np
import pynca as nca

print(f"pyNCA version: {nca.__version__}")

## 1. Load Data

The theophylline dataset contains concentration-time data from 6 subjects who received oral theophylline.

In [ ]:
# Load theophylline data
df = pd.read_csv("../tests/data/theophylline.csv")
print(f"Dataset shape: {df.shape}")
print(f"Subjects: {df['Subject'].unique()}")
df.head(12)

## 2. Create NCA Objects

In [ ]:
# Create concentration object
conc = nca.NCAConcentration(
    data=df,
    conc_col='conc',
    time_col='Time',
    subject_col='Subject'
)

# Create dose object
dose_df = df[['Subject', 'Dose']].drop_duplicates()
dose_df['Time'] = 0

dose = nca.NCADose(
    data=dose_df,
    dose_col='Dose',
    time_col='Time',
    subject_col='Subject'
)

# Combine into NCAData
data = nca.NCAData(conc=conc, dose=dose)
print(data)

## 3. Run NCA Analysis

In [ ]:
# Run NCA
results = nca.run_nca(data)

# View results in wide format
df_results = results.to_dataframe(wide=True)
df_results

## 4. Visualize Concentration-Time Data

In [ ]:
import matplotlib.pyplot as plt

# Plot all subjects on one graph
fig, ax = plt.subplots(figsize=(10, 6))
ax = nca.plot_conc_time(data, ax=ax)
ax.set_title('Theophylline Concentration-Time Profiles')
plt.tight_layout()
plt.show()

In [ ]:
# Plot each subject separately with semi-log scale
fig = nca.plot_conc_time_by_subject(data, ncols=3, log_y=True)
fig.suptitle('Individual Concentration-Time Profiles (Semi-log)', y=1.02)
plt.tight_layout()
plt.show()

## 5. Lambda_z Regression Diagnostic

In [ ]:
# Show lambda_z regression for Subject 1
subj1 = df[df['Subject'] == 1]
conc_arr = subj1['conc'].values
time_arr = subj1['Time'].values

fig, ax = plt.subplots(figsize=(8, 6))
ax = nca.plot_lambda_z(conc_arr, time_arr, ax=ax, title='Subject 1 - Lambda_z Regression')
plt.tight_layout()
plt.show()

## 6. Summary Statistics

In [ ]:
# Get summary statistics
summary = results.summary()
summary

In [ ]:
# Visualize parameter distributions
fig = nca.plot_parameter_summary(
    results, 
    parameters=['cmax', 'tmax', 'auc.last', 'half.life'],
    plot_type='box'
)
plt.show()

## 7. Forest Plot

In [ ]:
# Create forest plot for Cmax
ax = nca.plot_forest(results, 'cmax', ci_level=0.95)
plt.tight_layout()
plt.show()

## 8. Bootstrap Confidence Intervals

In [ ]:
from pynca.summary.summarize import bootstrap_summary

# Calculate bootstrap 95% CI for key parameters
boot_summary = bootstrap_summary(
    results,
    parameters=['cmax', 'auc.last', 'auc.inf.obs', 'half.life'],
    statistic='geomean',
    confidence=0.95,
    n_bootstrap=2000,
    random_state=42
)
boot_summary

## 9. Individual Parameter Calculations

In [ ]:
# Calculate partial AUC (e.g., AUC from 0 to 12 hours)
from pynca.calc.auc import calc_auc_partial

subj1 = df[df['Subject'] == 1]
auc_0_12 = calc_auc_partial(
    subj1['conc'].values,
    subj1['Time'].values,
    start=0,
    end=12
)
print(f"Subject 1 AUC(0-12h): {auc_0_12:.2f}")

In [ ]:
# Calculate time above threshold (e.g., MIC = 5 mg/L)
from pynca.calc.half_life import calc_time_above_threshold

time_above_mic = calc_time_above_threshold(
    subj1['conc'].values,
    subj1['Time'].values,
    threshold=5.0
)
print(f"Subject 1 Time above MIC (5 mg/L): {time_above_mic:.2f} hours")

## 10. Export Results

In [ ]:
# Export to CSV
results.to_csv("theophylline_nca_results.csv")
print("Results exported to theophylline_nca_results.csv")

# Export wide format
df_wide = results.to_dataframe(wide=True)
df_wide.to_csv("theophylline_nca_results_wide.csv", index=False)
print("Wide format exported to theophylline_nca_results_wide.csv")

## Summary

This notebook demonstrated:

1. Loading concentration and dose data
2. Creating NCA data structures
3. Running NCA analysis
4. Visualizing concentration-time profiles
5. Lambda_z regression diagnostics
6. Summary statistics
7. Forest plots
8. Bootstrap confidence intervals
9. Partial AUC and time above threshold calculations
10. Exporting results